<a href="https://colab.research.google.com/github/yueop/AS_LAB/blob/main/statistics_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import sys
from google.colab import drive

# 1. 드라이브 마운트
drive.mount('/content/drive')

# 2. 경로 설정
PROJECT_PATH = '/content/drive/MyDrive/AS_LAB'
if not os.path.exists(PROJECT_PATH):
    os.makedirs(PROJECT_PATH)

# 3. 시스템 경로 추가 및 작업 디렉토리 변경
if PROJECT_PATH not in sys.path:
    sys.path.append(PROJECT_PATH)
os.chdir(PROJECT_PATH)

print(f"현재 위치: {os.getcwd()}")

Mounted at /content/drive
현재 위치: /content/drive/MyDrive/AS_LAB


In [ ]:
# 파일 이름을 명시적으로 지정
FILE_NAME = "run_stats.py"
# 전체 경로 사용
SAVE_PATH = f"/content/drive/MyDrive/AS_LAB/{FILE_NAME}"
content = """
import yaml
import numpy as np
import os
import torch
import train
import eval as eval_script
from importlib import reload
import time

def run_statistics(config_name='config.yaml', num_trials=5):
    # 1. 설정 로드
    with open(config_name, 'r') as f:
        base_config = yaml.safe_load(f)

    project_name = base_config['project_name']
    print(f"=== [{project_name}] 실험 결과 집계 시작 ({num_trials}회 반복) ===")

    # 시드 설정 (재현성을 위해 고정된 시드 사용)
    seeds = [42, 100, 2026, 7, 0]
    if num_trials > 5:
        seeds += [np.random.randint(10000) for _ in range(num_trials - 5)]

    # 결과 저장소
    results = {
        'time': [],      # 학습 시간
        'accuracy': [],  # 정확도
        'precision': [], # 정밀도
        'recall': [],    # 재현율
        'f1_score': []   # F1 점수
    }

    # 2. 반복 실험 시작
    for i in range(num_trials):
        seed = seeds[i]
        print()
        print(f">> [Run {i+1}/{num_trials}] Seed: {seed}")

        # 설정 파일에 시드 업데이트
        base_config['train']['seed'] = seed
        with open('config.yaml', 'w') as f:
            yaml.dump(base_config, f)

        try:
            # [학습 실행] train.py가 반환한 시간을 받음
            reload(train)
            train_time = train.main()
            results['time'].append(train_time)

            # [평가 실행] eval.py가 반환한 지표들을 받음
            reload(eval_script)
            acc, prec, rec, f1 = eval_script.main()

            results['accuracy'].append(acc)
            results['precision'].append(prec)
            results['recall'].append(rec)
            results['f1_score'].append(f1)

            # 시드별 결과 즉시 출력
            print(f"   └─ Time: {train_time:.2f}s | Acc: {acc*100:.2f}% | F1: {f1:.4f}")

        except Exception as e:
            print(f"   └─ 에러 발생: {e}")
            import traceback
            traceback.print_exc()

    # 3. 최종 통계 계산 및 표 출력
    print()
    print("="*65)
    print(f" 📊 [최종 실험 결과 리포트] {project_name}")
    print("="*65)
    print(f"{'Metric':<15} | {'Mean':<12} | {'Std Dev':<10} | {'Best (Min/Max)':<15}")
    print("-" * 65)

    def print_stat(name, data, unit="", is_time=False):
        if len(data) == 0:
            print(f"{name:<15} | 데이터 없음")
            return

        mean_val = np.mean(data)
        std_val = np.std(data)

        if is_time: # 시간은 짧을수록 좋음 (Min)
            best_val = np.min(data)
            scale = 1
        else: # 성능은 높을수록 좋음 (Max)
            best_val = np.max(data)
            scale = 100 if unit == "%" else 1

        # 출력 포맷팅
        print(f"{name:<15} | {mean_val*scale:.2f}{unit:<6} | {std_val*scale:.2f}{unit:<5} | {best_val*scale:.2f}{unit}")

    # 성능 지표 출력
    print_stat("Accuracy", results['accuracy'], unit="%")
    print_stat("Precision", results['precision'], unit="%")
    print_stat("Recall", results['recall'], unit="%")
    print_stat("F1-Score", results['f1_score'], unit="") # F1은 소수점으로 표기
    print("-" * 65)
    # 시간 지표 출력
    print_stat("Train Time", results['time'], unit="s", is_time=True)
    print("="*65)

if __name__ == "__main__":
    if os.path.exists('config.yaml'):
        run_statistics()
    else:
        print("오류: config.yaml 파일이 없습니다. 설정 파일을 먼저 복사해주세요.")
"""
try:
    with open(SAVE_PATH, 'w', encoding='utf-8') as f:
        f.write(content.strip())
    print(f"파일이 안전하게 생성되었습니다: {SAVE_PATH}")
except Exception as e:
    print(f"저장 실패: {e}")

파일이 안전하게 생성되었습니다: /content/drive/MyDrive/AS_LAB/run_stats.py
